# Lab 3.3 IO-Link data collection using JSON interface

## 3.1 Python programming for data collection from IO-Link master

The entire schematic of the data communication using JSON interface using REST API for the IO-Link vibraton device from the IO-Link master is illustrated in Figure 12. As the power meter data collection, your computers (laptop and Raspberry Pi) are on the router network. The IO-Link master is also on the same router network with a static IP address. The ethernet cable (RJ45) is used for connection between the router and the IO-Link master. Note that the IO-Link vibration sensor is connected to the **port 1** of the master. In this case using REST API, each of your computer is not a master but client. The IO-Link master is a server. The data communication sequence to read data from the IO-Link master is summarized as follows.

1.  The computer (Raspberry Pi or laptop) requests data using REST API's post method to the IO-Link master 
2.  In the JSON body, specify "code", "cid", "address", "data".
3.  The IO-Link master read the IO-Link sensor data by the IO-Link protocol
4.  The IO-Link master sends the received data from the IO-Link sensor to the computer as the JSON format.
5.  The computer receives the JSON data and parse it.

To do this, we will use Python and 'requests' and 'json' packages which are the basic packages in Python. Moreover, refer to details of JSON interface of the IFM IO-Link master [here](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab3/IFM_JSON_Integration_for_IO-Link.pdf?raw=true). The IO-Link vibration sensor (VVB001) manual is also available [here](https://github.com/Eunseob/purdue_me597/blob/main/lab/lab3/IFM_VVB001_IO-Link_Interface_Description.pdf?raw=true). **Please note that this lab manual skips all details to extract data for your convenience. If you perform data collection from other IoT devices, the first thing you have to do is to read the manual and figure out data communication protocols and take appropriate action.**


<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/lab3_fig13.png?raw=true" width="60%">


*Figure 14 Haas CNC controller HMI: (A) main control buttons and hand jog, (B) display panel, and (C) control buttons*

For the data collection example, while the machine is running, we will run a G-code even if it is a simulation within the controller because we do not have a connected machine. If you are not familiar with G-code, please visit the website (https://www.cnccookbook.com/cnc-programming-g-code/, available on Feb. 4, 2022) or refer to Chapter 7: G-codes (p. 271) of the CNC controller manual (english---mill-ngc---operator's-manual---2019.pdf, on Brightspace). The machine runs as simulating a vertical 3-axis mill (VF series). The example G-code 
(O00597.nc, uploaded on Brightspace) is a face mill of a workpiece. The schematic of the G-code is illustrated in Figure 15. The figure is the top side view (+z-axis view) from the workpiece. Symbols of the schematic are shown on the right-hand side of the figure. This G-code is cut down the entire face of the workpiece in 10 mm depth with a 5 mm diameter end-mill of the z-axis. From the cutting condition (federate), and tool path, we can calculate how long the G-code takes. The entire will take around 10 minutes 20 seconds. TA will repeat running the same G-code throughout the lab. Even though the machine hardware is not connected to the CNC controller, it runs G-code and simulates the cutting. The purpose of this section is to collect CNC controller data via TCP/IP protocol while the G-code is running. 

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure15.png?raw=true)

*Figure 15 Schematic of example G-code (O00597.nc)*

---

<sup>1</sup>The Haas CNC controller HMI is supported by IN-MaC (Indiana Next Generation Manufacturing Competitiveness Center). 

## 3.2 CNC controller data collection via TCP/IP: Telnet using PuTTY

The Haas controller uses a TCP server to communicate over networks. On the remote computer, you can use any terminal program that supports TCP. In this part, we will read CNC controller data using PuTTY on your laptop. The Haas controller’s MDC (Machine Data Collection) lets you use Q command to extract data from the control through the ethernet port or the wireless networking option. In this lab, we will use the Wifi of Haas controller to be on the router network. The entire schematic for communication between computers and the CNC controller using TCP/IP is shown in Figure 16. Different from Modbus protocol, the computers are called clients, and the target machine (CNC controller in this case) is called server. It is possible that multiple clients can request the data from a TCP server. For the details of TCP/IP protocol, please refer to the IBM documentation (https://www.ibm.com/docs/en/aix/7.2?topic=protocol-tcpip-protocols, available on Feb. 4, 2022). 

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure16.png?raw=true)

*Figure 16 Schematic of communication for CNC controller*

Let’s collect data via Telnet using PuTTY. The MDC (Machine Data Collection) queries are shown in Table 6 (page 471 of Haas controller manual). The selected macro variables are shown in Table 7. The entire macro variable table is in 6.13.7 Macro Variables Table starting from page 231 of the Haas CNC manual. 

*Table 6 MDC queries and commands*

<table width="677">
<tbody>
<tr>
<td width="76">
<p>Command</p>
</td>
<td width="353">
<p>Definition</p>
</td>
<td width="248">
<p>Example Response</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q100</p>
</td>
<td width="353">
<p>Machine Serial Number</p>
</td>
<td width="248">
<p>SERIAL NUMBER, 1234567</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q101</p>
</td>
<td width="353">
<p>Control Software Version</p>
</td>
<td width="248">
<p>SOFTWARE VERSION, 100.17.000.2037</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q102</p>
</td>
<td width="353">
<p>Machine Model Number</p>
</td>
<td width="248">
<p>MODEL, VF</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q104</p>
</td>
<td width="353">
<p>Mode (LIST PROG, MDI, etc.)</p>
</td>
<td width="248">
<p>MODE, MEM</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q200</p>
</td>
<td width="353">
<p>Tool Changes (total)</p>
</td>
<td width="248">
<p>TOOL CHANGES, 35</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q201</p>
</td>
<td width="353">
<p>Tool Number in use</p>
</td>
<td width="248">
<p>USING TOOL, 4</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q300</p>
</td>
<td width="353">
<p>Power-on Time (total)</p>
</td>
<td width="248">
<p>P.O. TIME, 06282:17:13</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q301</p>
</td>
<td width="353">
<p>Motion Time (total)</p>
</td>
<td width="248">
<p>C.S. TIME, 00098:18:29</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q303</p>
</td>
<td width="353">
<p>Last Cycle Time</p>
</td>
<td width="248">
<p>LAST CYCLE, 00000:00:13</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q304</p>
</td>
<td width="353">
<p>Previous Cycle Time</p>
</td>
<td width="248">
<p>PREV CYCLE, 00000:00:01</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q402</p>
</td>
<td width="353">
<p>M30 Parts Counter #1 (resettable at control)</p>
</td>
<td width="248">
<p>M30 #1, 380</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q403</p>
</td>
<td width="353">
<p>M30 Parts Counter #2 (resettable at control)</p>
</td>
<td width="248">
<p>M30 #2, 380</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q500</p>
</td>
<td width="353">
<p>Three-in-one (PROGRAM, Oxxxxx, STATUS, PARTS, xxxxx)</p>
</td>
<td width="248">
<p>PROGRAM, MDI, IDLE, PARTS, 380</p>
</td>
</tr>
<tr>
<td width="76">
<p>?Q600</p>
</td>
<td width="353">
<p>Read a macro or system variable</p>
</td>
<td width="248">
<p>MACRO, 0.0</p>
</td>
</tr>
</tbody>
</table>

<br></br>
*Table 7 Selected macro variables of Haas CNC controller*

<table width="393">
<tbody>
<tr>
<td width="96">
<p>Variable</p>
</td>
<td width="297">
<p>Description</p>
</td>
</tr>
<tr>
<td width="96">
<p>#1064</p>
</td>
<td width="297">
<p>X axis maximum axis loads</p>
</td>
</tr>
<tr>
<td width="96">
<p>#1065</p>
</td>
<td width="297">
<p>Y axis maximum axis loads</p>
</td>
</tr>
<tr>
<td width="96">
<p>#1066</p>
</td>
<td width="297">
<p>Z axis maximum axis loads</p>
</td>
</tr>
<tr>
<td width="96">
<p>#1098</p>
</td>
<td width="297">
<p>Spindle load with Haas vector drive (read only)</p>
</td>
</tr>
<tr>
<td width="96">
<p>#3027</p>
</td>
<td width="297">
<p>Spindle RPM (read only)</p>
</td>
</tr>
<tr>
<td width="96">
<p>#5041</p>
</td>
<td width="297">
<p>X axis position (work coordinate)</p>
</td>
</tr>
<tr>
<td width="96">
<p>#5042</p>
</td>
<td width="297">
<p>Y axis position (work coordinate)</p>
</td>
</tr>
<tr>
<td width="96">
<p>#5043</p>
</td>
<td width="297">
<p>Z axis position (work coordinate)</p>
</td>
</tr>
</tbody>
</table>

### Task 3.1

Capture the result of checking network connection using ping test from Command Prompt on laptop to the Haas controller as Figure 11 and attach it to the report. 

* Please note that the IP address of the Haas controller will be different from the example. 

The correct query format is ?Q###, where ### is the query number, terminated with a new line. For instance, if you requests query ?Q100, the response from the controller is the machine serial number. Responses from the control begin with > and end with /r/n. Successful queries return the name of the query, then the requested information, separated by commas. For example, a query of ?Q102 returns MODEL, XXX, where XXX is the machine model. The comma lets you treat the output as comma-separated variable (CSV) data. An unrecognized command returns a question mark followed by the unrecognized command; e.g., ?Q105 returns ?, ?Q105.  

In addition, you can request the contents of any macro or system variables with the ?Q600 command. The query ?Q600 xxx shows the contents of macro variable xxx on the remote computer. For example, the query ?Q600 3027 (see Table 7)returns MACRO, 5000, which means the current spindle speed is 5000 RPM. Other than macro variables in Table 7, tens of thousands macro and system variables are available (see 6.13.7 Macro Variables Table from page 231 of the Haas CNC manual.). By reading macro variables and MDC via TCP/IP protocol, you can build up a digital twin for the CNC machine. 

Let’s try to read data using PuTTY. Please follow the procedures below.

1.	Open ‘PuTTY’ on your laptop 
2.	Click ‘Connection’ in ‘Category’ 
3.	Check ‘Enable TCP keepalives (SO_KEEPALIVE option)’ box

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Image1.png?raw=true)

4.	Click ‘Session’ in ‘Category’ 
5.	Type Haas controller IP address in ‘Host Name (or IP address)’
* TA will let you know the IP address
6. Type 5028 in ‘Port’ 
* Normally, TCP port is 25 like SSH port is 22, but in this case, we will use port number 5028 for TCP.
7.	Click ‘Telnet’ of ‘Connection type’ 
8.	Type Lab4_Haas in ‘Saved Sessions’ 
9.	Click ‘Save’ button so that you can use the same connection next time 

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Image2.png?raw=true)

10.	Click ‘Open’ and then you will see the window as Figure 17. 

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure17_2.png?raw=true)

*Figure 17 PuTTY terminal window for Telnet*

Now, you can request a query to the Haas CNC controller. Try ?Q100 to see the Machine Serial Number. The return may be ?, ???? ????’?????????Q100 because this is the first time to request data. Try the same query again and then the return will be SERIAL NUMBER, 1234567, which means the Machine Serial Number is 1234566.The requests afterwards will work without error. Next, let’s request a query to read a macro variable. Try ?Q600 3027 to see Spindle RPM. The return will be MACRO, ####. #### means the spindle speed in RPM unit. 

---

**PuTTY - PuTTY terminal window**

```
?Q100
?Q600 3027
```

---

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure18.png?raw=true)

*Figure 18 PuTTY terminal window after requesting machine serial number and spindle RPM*

### Task 3.2

Requests all command queries in Table 6 and then capture the PuTTY terminal window as Figure 18 and attach it to the report. 
* You do not need to request macro variables but try some queries to read macro variables for practice. 

By using Telnet of PuTTY, you can request data from Haas CNC controller before starting programming. It is an easy and light way to check network connections and test. This can be done on Raspberry Pi as well. Install PuTTY on Raspberry Pi and try. 

---

**Raspberry Pi - Terminal**

```
sudo apt install putty -y # This command is to install PuTTY. 
putty # This command is to run PuTTY. 
```

---

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure19.png?raw=true)

*Figure 19 Running PuTTY on Raspberry Pi using putty command (left) and Raspberry Pi PuTTY GUI*

### 3.3 Python programming for data collection from CNC controller

The limitation of data requests using PuTTY is that it is difficult to store data. In this part, we will request data using Python programming with socket package. The socket programming is a way of connecting two nodes on a network to communicate with each other. Sockets may be implemented over various channels such as Unix domain sockets, TCP, UDP, HTTP, and so on. If you are interested in Python network programming using socket package, visit the tutorial (https://www.tutorialspoint.com/python3/python_networking.htm, available on Feb. 5, 2022). The sample code 2 (lab4_sample2.py on Brightspace) is given as below. This code is to print out a timestamp and the measured values of serial number and spindle speed as Figure 20. Please refer to the comments on the script for the descriptions. 

---

**Python - Python3 (lab4_sample2.py)**

```
import socket
import datetime

# define the CNC controller IP (host) and port)
host = "192.168.1.x" # TA will let you know the IP address of the 
port = 5028

# create a socket object as s
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Initiate TCP server connection from CNC controller
s.connect((host, port))

"""
haasTCP requires an argument, query command as string data type.
This method returns response and data. Both are string data type.
Usage: hassTCP[0] returns the entire response of the query request.
       haasTCP[1] returns only data.
ex) haasTCP("?Q600 3027") returns spindle speed.
"""
def haasTCP(query:str):
    query += "\r\n"
    query_enc = query.encode('utf-8')
    s.sendall(query_enc)
    b = b'\x00'
    response = ""
    while b != b'\n':
        b = s.recv(1)
        response += b.decode('utf-8')
    data = response.split(",")[-1].replace("\r\n","").replace(" ","")
    return response, data

now = datetime.datetime.now()

# request serial number and spindle speed
serial_number = haasTCP("?Q100")[1]
spindle_speed = haasTCP("?Q600 3027")[1]

# print out timestamp and collected data.
print("{}: Serial number is {} and Spindle speed is {} RPM".format(now,serial_number,spindle_speed))

# gently close the socket object. This should be at the end of the script
s.close()


```

---



### Task 3.3

Run the sample code 2 and then capture and attach the result as Figure 20. 
* Capture either one of terminal window as Figure 20 (top) or Thonny IDE Shell as Figure 20 (bottom). ※ Try this on your laptop and see if it works as well. 

![picture](https://github.com/hewp84/tinyml/blob/main/img/L4_Figure20.png?raw=true)

*Figure 20 Results of running sample code 2: terminal window (top) and Thonny IDE shell (right)*


### Task 3.4

1.	Modify sample code 2 to collect data for at least 5 minute and save data as a CSV file and upload the CSV file on Brightspace. 
* Set the sampling period 1 second. (Hint: use ‘while’ loop and ‘time’ package) 
* Data (header) should include time and the queries below. 
  * ?Q104 (Mode) 
  * ?Q201 (Tool number in use) 
  * ?Q300 (Power-on Time) 
  * ?Q301 (Motion Time) 
  * ?Q304 (Previous Cycle Time) 
  * ?Q402 (M30 Parts Counter #1) 
  * ?Q600 1098 (Spindle load with Haas vector drive) 
  * ?Q600 3027 (Spindle RPM) 
  * ?Q600 5041 (X axis position) 
  * ?Q600 5042 (Y axis position) 
  * ?Q600 5043 (Z axis position) 
2.	Plot tool path in 3D using 3-axis position data and attach it to the report. 


###Analyze power meter data using the CSV file from Task 3.4

In [ ]:
#@title 1) What is average value of each measured data? Were all values as expected? Explain.<br>Consider the standard electricity and specification of the vacuum pump. {display-mode: "form"}
T1 = '' #@param {type:"string"}
print(T1)

In [ ]:
#@title 2) Calculate averaged reactive and apparent power. {display-mode: "form"}
T2 = '' #@param {type:"string"}
print(T2)

In [ ]:
#@title 2) Calculate averaged reactive and apparent power. {display-mode: "form"}
T2 = '' #@param {type:"string"}
print(T2)

In [ ]:
#@title 3) Is the CSV file efficient for collecting CNC controller data? {display-mode: "form"}
T3 = '' #@param {type:"string"}
print(T3)

In [ ]:
#@title 3) Is the CSV file efficient for collecting CNC controller data? {display-mode: "form"}
T3 = '' #@param {type:"string"}
print(T3)

In [ ]:
#@title 3a) In the CSV file data of the CNC controller by Task 3.4, which data are EVENT and SAMPLE, respectively?<br>   *EVENT stands for discrete data when any change regardless of sampling period.<br>   *SAMPLE stands for continuous measurable data. {display-mode: "form"}
T3a = '' #@param {type:"string"}
print(T3a)

In [ ]:
#@title 3b) If CSV file is inefficient, how can we improve the data collection strategy? {display-mode: "form"}
T3b = '' #@param {type:"string"}
print(T3b)

In [ ]:
#@title 4) To calculate OEE of the CNC machine, which data is needed? {display-mode: "form"}
T4 = '' #@param {type:"string"}
print(T4)

###Task 3.5

<ol>
<li> Model Unit 4 (prelab + lab) by creating a flow diagram that illustrates the logic, sequence, and processes of the unit.
  <ol type="a"> <li> <hr> Place URL here <hr> </li> </li> </ol>

